Imports:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, ConcatDataset
import pandas as pd
import time
import itertools
import random
import json
import os
import numpy as np
from pathlib import Path

Saving files in case runtime disconnects. This code will:
1. Check if the code is being run in Google Colab
- If so, it will try to mount your google drive
- Then it will save the files later to a directory called ml_experiment_results in MyDrive
2. If the code is not being run in Google Colab, it will be saved to your pc on a local directory called model_results

This is just to save the output tables so that they aren't lost if the runtime suddenly disconnects


In [ ]:
def get_save_directory():
    try:
        # Check if code is running in Google Colab
        import google.colab
        from google.colab import drive

        # Try to mount Drive (should skip if already mounted)
        try:
            drive.mount('/content/drive', force_remount=False)
        except:
            pass

        # Using a standard path in Drive that should work for everyone
        save_dir = Path('/content/drive/MyDrive/ml_experiment_results')
        print(f"Running in Colab - saving to Google Drive: {save_dir}")
        return save_dir
    except ImportError:
        # If not in Colab, use local directory
        save_dir = Path.cwd() / "model_results"
        print(f"Running locally - saving to: {save_dir}")
        return save_dir

Initializing Random_seed, architectures for both MLP and CNNs, and Hyperparameter Values
- Note: Setting all random seeds to 73 for reproducibility

In [ ]:
def set_all_seeds(seed=73):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_all_seeds(73)

In [ ]:
CONFIG = {
    'random_seed': 73,
    'max_epochs': 20,
    'early_stop_patience': 3,
    'param_samples': 10,
    'device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    'save_dir': get_save_directory()
}

MLP_ARCHITECTURES = {
    "shallow": [128],
    "medium": [512, 156, 128],
    "deep": [1024, 512, 256, 128, 64]
}

CNN_ARCHITECTURES = {
    "baseline": "baseline_cnn",
    "enhanced": "enhanced_cnn",
    "deeper": "deeper_cnn"
}

PARAM_SPACE = list(itertools.product(
    [0.01, 0.001, 0.0001],  # learning rates
    [32, 64, 128],          # batch sizes
    ["Adam", "SGD"],        # optimizers
    [0.2, 0.5]              # dropouts
))

Mounted at /content/drive
Running in Colab - saving to Google Drive: /content/drive/MyDrive/ml_experiment_results


Importing MNIST and CIFAR-10 datasets for both models:

In [ ]:
def load_datasets(use_cnn=False):
    # Default is not cnn (MLP) If it is for CNN, we normalize it (don't flatten it yet)
    if use_cnn:
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
    # Here we flatten it for the MLP model
    else:
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x.view(-1))
        ])

    # MNIST Dataset import - 50k training, 10k validation split
    mnist_full = datasets.MNIST('mnist_data', train=True, download=True, transform=transform)
    mnist_test = datasets.MNIST('mnist_data', train=False, download=True, transform=transform)
    mnist_train, mnist_val = random_split(mnist_full, [50000, 10000])

    # Again, if it's for CNN, we normalize, this time for 3 channels since these images are RGB (not grayscale)
    if use_cnn:
        cifar_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    # If it's MLP we flatten it as before
    else:
        cifar_transform = transform

    # CIFAR Dataset import - 45k training, 5k validation split
    cifar_full = datasets.CIFAR10('cifar_data', train=True, download=True, transform=cifar_transform)
    cifar_test = datasets.CIFAR10('cifar_data', train=False, download=True, transform=cifar_transform)
    cifar_train, cifar_val = random_split(cifar_full, [45000, 5000])

    # Input dimensions for MNIST and CIFAR (to be used later)
    mnist_shape = (1, 28, 28) if use_cnn else 784
    cifar_shape = (3, 32, 32) if use_cnn else 3072

    return {
        'mnist': {'train': mnist_train, 'val': mnist_val, 'test': mnist_test, 'input_dim': mnist_shape},
        'cifar10': {'train': cifar_train, 'val': cifar_val, 'test': cifar_test, 'input_dim': cifar_shape}
    }

Defining base architecture for MLP:


In [ ]:
# Basic MLP to work for all 3 architecture types
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_layers, dropout=0.0, num_classes=10):
        super(MLP, self).__init__()
        layers = []
        prev_dim = input_dim
        for h in hidden_layers:
            layers.append(nn.Linear(prev_dim, h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = h
        layers.append(nn.Linear(prev_dim, num_classes))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

Defining 3 Architectures for CNNs

In [ ]:
class BaselineCNN(nn.Module):
    def __init__(self, in_channels, dropout=0.0, num_classes=10):
        super(BaselineCNN, self).__init__()
        # 2 convolutional layers, chose kernel = 3, padding = 1 for simplicity
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # + max pooling
        self.pool = nn.MaxPool2d(2, 2)

        self.fc_input_size = None
        self.fc = None
        self.dropout = nn.Dropout(dropout)
        self.num_classes = num_classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Dynamically creates the fully connected layer on the first pass
        if self.fc is None:
            self.fc_input_size = x.shape[1] * x.shape[2] * x.shape[3]
            self.fc = nn.Linear(self.fc_input_size, self.num_classes).to(x.device)

        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x


In [ ]:
class EnhancedCNN(nn.Module):
  # Chose same parameters as before (kernel = 3, padding = 1), juts added batch normalization each time & dropout
    def __init__(self, in_channels, dropout=0.0, num_classes=10):
        super(EnhancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout)

        self.fc_input_size = None
        self.fc = None
        self.num_classes = num_classes

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.dropout(x)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)

        if self.fc is None:
            self.fc_input_size = x.shape[1] * x.shape[2] * x.shape[3]
            self.fc = nn.Linear(self.fc_input_size, self.num_classes).to(x.device)

        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x


In [ ]:
class DeeperCNN(nn.Module):
  # Same as before, just added more layers of batch normalization and dropout, same kernel size and padding
    def __init__(self, in_channels, dropout=0.0, num_classes=10):
        super(DeeperCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout)

        self.fc_input_size = None
        self.fc1 = None
        self.fc2 = None
        self.num_classes = num_classes

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.dropout(x)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout(x)

        if self.fc1 is None:
            self.fc_input_size = x.shape[1] * x.shape[2] * x.shape[3]
            self.fc1 = nn.Linear(self.fc_input_size, 256).to(x.device)
            self.fc2 = nn.Linear(256, self.num_classes).to(x.device)

        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


Model Training Functions

In [ ]:
# Helper function to get the validation accuracy for a given model
def evaluate_model(model, dataloader, device, is_cnn = False):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            # Only flatten for MLP
            if not is_cnn and len(images.shape) > 2:
                images = images.view(images.size(0), -1)

            outputs = model(images)
            predicted = outputs.argmax(dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    return correct / total

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, device, criterion=nn.CrossEntropyLoss(),
early_stop_patience=3, max_epochs=20, is_cnn=False, fixed_epochs=None):
    model.to(device)
    best_val_acc = 0.0
    patience_counter = 0
    start_time = time.time()
    best_epoch = 0
    best_state = None
    # Training each model for maximum of 20 epochs
    # Added later: to avoid data leakage, when we retrain on the training + validation set,
    # we use the same number of epochs that we found in the hyperparameter tuning
    epochs_to_run = fixed_epochs if fixed_epochs else max_epochs
    for epoch in range(epochs_to_run):
        model.train()
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(X)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

        if not fixed_epochs:
            val_acc = evaluate_model(model, val_loader, device, is_cnn)
            print(f"Epoch {epoch+1}: Validation Accuracy = {val_acc:.4f}")
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_epoch = epoch + 1
                patience_counter = 0
                best_state = model.state_dict()
            else:
                patience_counter += 1
                if patience_counter >= early_stop_patience:
                    print(f"Early stopping at epoch {epoch - 2}")
                    best_epoch = epoch - 2
                    break

    runtime = time.time() - start_time
    if fixed_epochs:
        best_val_acc = evaluate_model(model, val_loader, device, is_cnn)
        best_epoch = fixed_epochs
    elif best_state:
        model.load_state_dict(best_state)
    return best_val_acc, runtime, best_epoch

In [ ]:
# Helper function for optimizers, uses momentum = 0.9 for SGD (decided not to tune this hyperparameter)
def create_optimizer(opt_name, model_params, lr):
    if opt_name == "Adam":
        return optim.Adam(model_params, lr=lr)
    else:
        return optim.SGD(model_params, lr=lr, momentum=0.9)

In [ ]:
# Helper function that creates the model type for each architecture from the name by calling previously defined functions
def create_model(model_type, arch_name, input_dim, dropout):
    if model_type == "MLP":
        hidden_layers = MLP_ARCHITECTURES[arch_name]
        return MLP(input_dim=input_dim, hidden_layers=hidden_layers, dropout=dropout)
    else:  # CNN
        in_channels = input_dim[0]  # 1 for MNIST, 3 for CIFAR
        if arch_name == "baseline":
            return BaselineCNN(in_channels=in_channels, dropout=dropout)
        elif arch_name == "enhanced":
            return EnhancedCNN(in_channels=in_channels, dropout=dropout)
        else:  # deeper
            return DeeperCNN(in_channels=in_channels, dropout=dropout)

Hyperparameter Tuning

In [ ]:
# Combines all previous functions to get ideal hyperparameters
def run_experiments(dataset_name, input_dim, train_set, val_set, config, model_type="MLP"):
    random.seed(config['random_seed'])
    sampled_params = random.sample(PARAM_SPACE, config['param_samples'])
    results = []

    architectures = MLP_ARCHITECTURES if model_type == "MLP" else CNN_ARCHITECTURES
    is_cnn = (model_type == "CNN")

    for arch_name in architectures.keys():
        for lr, batch, opt_name, drop in sampled_params:
            # Setup data loaders
            train_loader = DataLoader(train_set, batch_size=batch, shuffle=True)
            val_loader = DataLoader(val_set, batch_size=batch, shuffle=False)

            # Create model and optimizer
            model = create_model(model_type, arch_name, input_dim, drop)
            optimizer = create_optimizer(opt_name, model.parameters(), lr)

            # Train
            print(f"\n{dataset_name} | {model_type} | {arch_name} | LR={lr} | Batch={batch} | Opt={opt_name} | Drop={drop}")
            val_acc, runtime, stop_epoch = train_model(
                model, train_loader, val_loader, optimizer,
                config['device'], max_epochs=config['max_epochs'],
                early_stop_patience=config['early_stop_patience'],
                is_cnn=is_cnn
            )

            print(f"Val Acc: {val_acc:.4f} | Runtime: {runtime:.2f}s | Stopped at epoch {stop_epoch}")

            results.append({
                "model_type": model_type,
                "architecture": arch_name,
                "learning_rate": lr,
                "batch_size": batch,
                "optimizer": opt_name,
                "dropout": drop,
                "stop_epoch": stop_epoch,
                "validation_accuracy": val_acc,
                "runtime": runtime
            })

    return results

Function for Getting the Test Accuracy on Models with Best Hyperparameters

In [ ]:
def retrain_and_test(best_config, train_set, val_set, test_set, input_dim, config, model_type="MLP"):
    full_train_set = ConcatDataset([train_set, val_set])
    full_train_loader = DataLoader(full_train_set, batch_size=int(best_config["batch_size"]), shuffle=True)
    test_loader = DataLoader(test_set, batch_size=int(best_config["batch_size"]), shuffle=False)

    model = create_model(model_type, best_config["architecture"], input_dim, best_config["dropout"])
    model.to(config['device'])

    optimizer = create_optimizer(best_config["optimizer"], model.parameters(), best_config["learning_rate"])

    is_cnn = (model_type == "CNN")
    fixed_epochs = best_config["stop_epoch"]

    print(f"Retraining for {fixed_epochs} epochs (no early stopping)...")
    start_time = time.time()

    model.train()
    for epoch in range(fixed_epochs):
        for X, y in full_train_loader:
            X, y = X.to(config['device']), y.to(config['device'])
            optimizer.zero_grad()
            out = model(X)
            loss = nn.CrossEntropyLoss()(out, y)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{fixed_epochs} complete")

    runtime = time.time() - start_time

    test_acc = evaluate_model(model, test_loader, config['device'], is_cnn)
    print(f"Test accuracy: {test_acc:.4f} | Runtime: {runtime:.2f}s")

    return test_acc, runtime

Functions to Output Final Table (and save results in case runtime disconnects)

In [ ]:
# Helper function to get best validation accuracy across all architectures
def get_best_config(results_list):
    return max(results_list, key=lambda x: x["validation_accuracy"])

Outputs the table given in the assignment

In [ ]:
def summarize_results(results_list, best_test_acc=None, best_test_runtime=None, model_type="MLP"):
    unique_archs = sorted(set(r["architecture"] for r in results_list))

    # Group by architecture and find best run per architecture
    grouped_results = {arch: [] for arch in unique_archs}
    for run in results_list:
        grouped_results[run["architecture"]].append(run)

    best_runs = {arch: max(runs, key=lambda x: x["validation_accuracy"])
                 for arch, runs in grouped_results.items()}

    best_overall = max(results_list, key=lambda x: x["validation_accuracy"])

    # Build summary with proper architecture names
    arch_names = [arch.title() for arch in unique_archs] + ["Best Overall"]

    summary = {
        "Model": [model_type] * (len(unique_archs) + 1),
        "Architecture": arch_names,
        "Learning Rate": [best_runs[arch]["learning_rate"] for arch in unique_archs] + [best_overall["learning_rate"]],
        "Batch Size": [best_runs[arch]["batch_size"] for arch in unique_archs] + [best_overall["batch_size"]],
        "Optimizer": [best_runs[arch]["optimizer"] for arch in unique_archs] + [best_overall["optimizer"]],
        "Dropout": [best_runs[arch]["dropout"] for arch in unique_archs] + [best_overall["dropout"]],
        "Stop Epoch": [best_runs[arch]["stop_epoch"] for arch in unique_archs] + [best_overall["stop_epoch"]],
        "Validation Accuracy": [best_runs[arch]["validation_accuracy"] for arch in unique_archs] + [best_overall["validation_accuracy"]],
        "Runtime (min)": [best_runs[arch]["runtime"]/60 for arch in unique_archs] + [best_test_runtime/60 if best_test_runtime else None],
        "Test Accuracy": [None] * len(unique_archs) + [best_test_acc]
    }

    return pd.DataFrame(summary)

Saves results to JSON files (to be saved in Google Drive/locally later)

In [ ]:
def save_results(results, filename, save_dir):
    save_dir.mkdir(parents=True, exist_ok=True)
    filepath = save_dir / filename
    with open(filepath, "w") as f:
        json.dump(results, f, indent=4)
    print(f"Saved to {filepath}")

def load_results(filename, save_dir):
    filepath = save_dir / filename
    if filepath.exists():
        with open(filepath, "r") as f:
            return json.load(f)
    return None

Calls on previous functions to run code for each model
- Also checks if the file already exists so I don't have to re-run code that has already been executed if runtime disconnects for any reason



In [ ]:
def run_model_experiments(datasets_dict, model_type, config):
    for dataset_name in ['mnist', 'cifar10']:
        data = datasets_dict[dataset_name]
        results_file = f"{dataset_name}_{model_type.lower()}_results.json"

        # Check if results already exist
        existing_results = load_results(results_file, config['save_dir'])
        if existing_results:
            print(f"Found existing results for {dataset_name} {model_type}, skipping experiments...")
            results = existing_results
        else:
            # Run experiments
            results = run_experiments(
                dataset_name.upper(),
                data['input_dim'],
                data['train'],
                data['val'],
                config,
                model_type=model_type
            )
            save_results(results, results_file, config['save_dir'])

        # Retrain best model and test
        best_config = get_best_config(results)
        print(f"\nRetraining best {model_type} model for {dataset_name.upper()}...")
        test_acc, test_runtime = retrain_and_test(
            best_config, data['train'], data['val'], data['test'],
            data['input_dim'], config, model_type=model_type
        )

        # Summarize and save
        df_summary = summarize_results(results, test_acc, test_runtime, model_type)
        summary_file = config['save_dir'] / f"{dataset_name}_{model_type.lower()}_table.csv"
        df_summary.to_csv(summary_file, index=False)
        print(df_summary)

Combines results into final 2 tables

In [ ]:
def combine_results(config):
    for dataset_name in ['mnist', 'cifar10']:

        # Load individual result tables
        mlp_table_path = config['save_dir'] / f"{dataset_name}_mlp_table.csv"
        cnn_table_path = config['save_dir'] / f"{dataset_name}_cnn_table.csv"

        combined_dfs = []

        if mlp_table_path.exists():
            df_mlp = pd.read_csv(mlp_table_path)
            combined_dfs.append(df_mlp)
        else:
            print("Error")

        if cnn_table_path.exists():
            df_cnn = pd.read_csv(cnn_table_path)
            combined_dfs.append(df_cnn)
        else:
            print("Error")

        if combined_dfs:
            # Combine and save
            df_combined = pd.concat(combined_dfs, ignore_index=True)
            combined_file = config['save_dir'] / f"{dataset_name}_combined_table.csv"
            df_combined.to_csv(combined_file, index=False)
            print(df_combined.to_string(index=False))
        else:
            print("Error")

Main function that runs all the code, should take roughly 5 hours to run

In [ ]:
print(f"Using device: {CONFIG['device']}")
print(f"Save directory: {CONFIG['save_dir']}")
set_all_seeds(CONFIG['random_seed'])

# Run MLP experiments
datasets_mlp = load_datasets(use_cnn=False)
run_model_experiments(datasets_mlp, "MLP", CONFIG)

Using device: cuda
Save directory: /content/drive/MyDrive/ml_experiment_results


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 496kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.70MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.0MB/s]
100%|██████████| 170M/170M [00:08<00:00, 19.3MB/s]



MNIST | MLP | shallow | LR=0.001 | Batch=64 | Opt=Adam | Drop=0.5
Epoch 1: Validation Accuracy = 0.9311
Epoch 2: Validation Accuracy = 0.9517
Epoch 3: Validation Accuracy = 0.9582
Epoch 4: Validation Accuracy = 0.9617
Epoch 5: Validation Accuracy = 0.9640
Epoch 6: Validation Accuracy = 0.9670
Epoch 7: Validation Accuracy = 0.9689
Epoch 8: Validation Accuracy = 0.9694
Epoch 9: Validation Accuracy = 0.9702
Epoch 10: Validation Accuracy = 0.9716
Epoch 11: Validation Accuracy = 0.9707
Epoch 12: Validation Accuracy = 0.9729
Epoch 13: Validation Accuracy = 0.9734
Epoch 14: Validation Accuracy = 0.9744
Epoch 15: Validation Accuracy = 0.9741
Epoch 16: Validation Accuracy = 0.9732
Epoch 17: Validation Accuracy = 0.9732
Early stopping at epoch 14
Val Acc: 0.9744 | Runtime: 128.22s | Stopped at epoch 14

MNIST | MLP | shallow | LR=0.01 | Batch=64 | Opt=SGD | Drop=0.5
Epoch 1: Validation Accuracy = 0.9202
Epoch 2: Validation Accuracy = 0.9397
Epoch 3: Validation Accuracy = 0.9507
Epoch 4: Validat

In [22]:
# Run CNN exper iments
datasets_cnn = load_datasets(use_cnn=True)
run_model_experiments(datasets_cnn, "CNN", CONFIG)



MNIST | CNN | baseline | LR=0.001 | Batch=64 | Opt=Adam | Drop=0.5
Epoch 1: Validation Accuracy = 0.9122
Epoch 2: Validation Accuracy = 0.9298
Epoch 3: Validation Accuracy = 0.9354
Epoch 4: Validation Accuracy = 0.9417
Epoch 5: Validation Accuracy = 0.9473
Epoch 6: Validation Accuracy = 0.9517
Epoch 7: Validation Accuracy = 0.9497
Epoch 8: Validation Accuracy = 0.9539
Epoch 9: Validation Accuracy = 0.9563
Epoch 10: Validation Accuracy = 0.9586
Epoch 11: Validation Accuracy = 0.9583
Epoch 12: Validation Accuracy = 0.9604
Epoch 13: Validation Accuracy = 0.9589
Epoch 14: Validation Accuracy = 0.9595
Epoch 15: Validation Accuracy = 0.9618
Epoch 16: Validation Accuracy = 0.9624
Epoch 17: Validation Accuracy = 0.9606
Epoch 18: Validation Accuracy = 0.9638
Epoch 19: Validation Accuracy = 0.9633
Epoch 20: Validation Accuracy = 0.9630
Val Acc: 0.9638 | Runtime: 272.27s | Stopped at epoch 18

MNIST | CNN | baseline | LR=0.01 | Batch=64 | Opt=SGD | Drop=0.5
Epoch 1: Validation Accuracy = 0.8879


In [25]:
# Combine results - creates one combined table per dataset
table = combine_results(CONFIG)
table_final = pd.DataFrame(table)
table_final

Model Architecture  Learning Rate  Batch Size Optimizer  Dropout  Stop Epoch  Validation Accuracy  Runtime (min)  Test Accuracy
  MLP         Deep           0.01          32       SGD      0.2          18               0.9836       3.396692            NaN
  MLP       Medium           0.01          32       SGD      0.2           8               0.9795       1.691885            NaN
  MLP      Shallow           0.01          32       SGD      0.2          20               0.9790       2.811602            NaN
  MLP Best Overall           0.01          32       SGD      0.2          18               0.9836       3.217230         0.9829
  CNN     Baseline           0.01          32       SGD      0.2          19               0.9758       4.960586            NaN
  CNN       Deeper           0.01          32       SGD      0.2          14               0.9930       4.677077            NaN
  CNN     Enhanced           0.01          32       SGD      0.2          18               0.9776       

""
